# googlemt
> Library to use Google's machine translation services

In [ ]:
#| default_exp googlemt

In [ ]:
#| hide
import os

In [ ]:
#| hide
running_in_colab = 'google.colab' in str(get_ipython())
if running_in_colab:
    from google.colab import drive
    drive.mount('/content/drive')
    homedir = "/content/drive/MyDrive"
else:
    homedir = os.getenv('HOME')

In [ ]:
#| hide
if running_in_colab:
    github_test_folder = homedir+"/github/polyglottech/mteval"
    %cd {github_test_folder}
    !pip3 install nbdev
    !pip3 install -e '.[dev]'

In [ ]:
#| hide
from dotenv import load_dotenv

In [ ]:
#| hide
if running_in_colab:
    # Colab doesn't have a mechanism to set environment variables other than python-dotenv
    env_file = homedir+'/secrets/.env'
    %load_ext dotenv
    %dotenv {env_file}

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import os
import six
from google.cloud import translate

class googletranslate:
    """Class to get translations from the Google Translate API"""

    def __init__(self):
        """ Constructor of googletranslate class. Load the credentials for the Google API"""
        # Google credentials need to be available in JSON file specified in the GOOGLE_APPLICATION_CREDENTIALS environment variable
        project_id = os.getenv('GOOGLE_PROJECT_ID')
        location = "global"
        
        self._translate_client = translate.TranslationServiceClient()
        self._parent = f"projects/{project_id}/locations/{location}"

    def translate_text(self,sourcelang, targetlang, text):
        """Function to translate text into the target language"""
        if isinstance(text, six.binary_type):
            text = text.decode("utf-8")

        response = self._translate_client.translate_text(
            request={
                "parent": self._parent,
                "contents": [text],
                "mime_type": "text/plain",  # mime types: text/plain, text/html
                "source_language_code": sourcelang,
                "target_language_code": targetlang,
            }
        )

        return response.translations[0].translated_text

In [ ]:
#| hide
# Current nbdev testing stomps on temporary Google credentials before testing can be run
# Therefore it isn't possible right now to test this in Github Actions
if not os.getenv('GITHUB_ACTIONS') == "true":
    gt = googletranslate()
    translation = gt.translate_text("en","it","This is a cool car!")
    assert translation == "Questa è una bella macchina!"

In [ ]:
show_doc(googletranslate.translate_text)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()